# Agentic fighters

## Exercise 1 - LangChain basics

### 1.1 - Chatbot simple

Bienvenidos a la primera parte del workshop!

Vamos a empezar viendo cómo podemos generar una aplicación basada en IA que utilice LangChain, un framework muy popular para desarrollar apps con LLMs.

#### Preparar las claves

Primero, vamos a preparar el entorno de Python para poder usar las claves de OpenAI. Hay que definir la OPENAI_API_KEY en el archivo .env.

El código busca la clave y fija unos parámetros:

- LLM_MODEL: el modelo a utilizar
- LLM_TEMPERATURE: parámetro que controla la aleatoriedad de las respuestas (0 significa que será completamente determinista)

In [ ]:
import os

In [ ]:
if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

print(os.environ.get("OPENAI_API_KEY"))

LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0

In [ ]:
from IPython.display import Markdown
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

Ahora vamos a crear el chabot con LangChain. Usaremos:

- ChatOpenAI: la interfaz a los modelos de OpenAI
- SystemMessage: define el comportamiento general del modelo
- HumanMessage: representa el input del usuario

Vamos a crear un chatbot con la temática deseada. Para ello:

1. instanciamos el modelo
2. definimos el system prompt que define el rol del chatbot
3. enviamos una query y recibimos una respuesta con el método .invoke()

Así vemos el patrón básico de interacciones: prompt → respuesta.

In [ ]:
# TODO: Create a ChatOpenAI instance with the LLM model and temperature
base_model = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)

In [ ]:
BASE_PROMPT = """
(Set system prompt)
"""

In [ ]:
# Request from the client
request = " (Request to be done) "

# Message list for the base model
messages = [
    SystemMessage(BASE_PROMPT),
    HumanMessage(request),
]
print(messages)

In [ ]:
# Invoke the model with the messages
response = base_model.invoke(messages)

In [ ]:
Markdown(response.content)

### 1.2 - Prompt template

LangChain proporciona formas de manejar los prompts, para ser consistente y con ello poder parsear resultados convenientemente.

In [ ]:
template_string = """Traduce el texto entre comillas simples al {target_language}:
'{input_text}'
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)
prompt_template

In [ ]:
prompt_template.messages[0].prompt

In [ ]:
prompt_template.messages[0].prompt.input_variables

In [ ]:
target_language = "English"
input_text = "Estamos haciendo un workshop en la UPV para aprender sobre agentes y poder desarrollar una app generativa de peleas!"

In [ ]:
message = prompt_template.format_messages(input_text=input_text, target_language=target_language)
print(type(message))
message

In [ ]:
message[0]

In [ ]:
response = base_model.invoke(message)
print(response)
Markdown(response.content)

### 1.3 - Parsers

Esto nos devuelve una respuesta del tipo _AIMessage_. Pero vamos a ver si lo que queremos es parsearla y que devuelva un JSON.

In [ ]:
import copy

prompt_template_json = copy.deepcopy(prompt_template)

In [ ]:
prompt_template_json.messages[0].prompt.template = f"{prompt_template.messages[0].prompt.template}. Crea la respuesta en formato JSON, con claves 'idioma_original' y 'traducido'"
prompt_template_json.messages[0].prompt.template

In [ ]:
message = prompt_template_json.format_messages(input_text=input_text, target_language=target_language)
response = base_model.invoke(message)
response

In [ ]:
response.content.get("traducido")

No se ha generado un JSON, sino que es un string con un formato similar.

Vamos a ver qué podemos hacer.

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [ ]:
response_schema = [
    ResponseSchema(name="idioma_original", description="The original text"),
    ResponseSchema(name="traducido", description="The translated text"),
]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schema)
output_parser

In [ ]:
format_intructions = output_parser.get_format_instructions()
Markdown(format_intructions)

In [ ]:
prompt_template_json_format = """Traduce el texto entre comillas simples al {target_language}:
'{input_text}'.

{format_intructions}
"""
prompt = ChatPromptTemplate.from_template(prompt_template_json_format)
message = prompt.format_messages(input_text=input_text, target_language=target_language, format_intructions=format_intructions)
print(type(message))
message

In [ ]:
response = base_model.invoke(message)
response

In [ ]:
output_dict = output_parser.parse(response.content)
output_dict

In [ ]:
output_dict.get("traducido")

### Extra: Chains -> LangChain Expression Language (LCEL)

Las cadenas eran el elemento básico de LangChain, que nos permiten ejecutar prompts uno detrás de otro (o con la estructura definida).

Hoy en día estan deprecadas. Son interesantes de conocer, pero ahora todo se basa en Runnables, que es lo que utiliza LangGraph. Pasamos directamente a ello.